In [4]:
print(super)

<class 'super'>


In [16]:
# 古い方法
class Mama:
    def says(self):
        print("Do your homework!!")

class Sister(Mama):
    def says(self):
        Mama.says(self)
        print("And, clean up your room!!")

In [17]:
Sister().says()

Do your homework!!
And, clean up your room!!


In [19]:
class Sister(Mama):
    def says(self):
        super(Sister,self).says()
        print("And, clean up your room!!")

In [20]:
Sister().says()

Do your homework!!
And, clean up your room!!


In [21]:
class Sister(Mama):
    def says(self):
        super().says()# 省略形
        print("And, clean up your room!!")

In [22]:
Sister().says()

Do your homework!!
And, clean up your room!!


In [23]:
# メソッドの外で使うときは引数が必要
anita = Sister()
super(anita.__class__,anita).says()
## super 二つ目の引数を省略するとインスタンスに束縛されていない型を返す

Do your homework!!


In [35]:
# 引数なしの場合でもclassmemethodsデコレータのついたメソッドから利用できる
class Pizza:
    def __init__(self,toppings):
        self.toppings = toppings
        
    # クラスの有効な出力結果を返す特殊メソッド
    def __repr__(self):
        return "と".join(self.toppings)+"がトッピングされたピザ"
    
    @classmethod
    def recommend(cls):
        """いくつかのトッピングが乗せられたオス勧めピザの紹介"""
        return cls(["スパム","ハム","卵"])

class VikingPizza(Pizza):
    @classmethod
    def recommend(cls):
        """親クラスと同じようなお勧めピザだが，ハラペーニョ大量に追加"""
        recommended = super().recommend()
        recommended.toppings += ["ハラペーニョ"]*5
        return recommended
    
        
    

In [36]:
print(Pizza.recommend())
print(VikingPizza.recommend())

スパムとハムと卵がトッピングされたピザ
スパムとハムと卵とハラペーニョとハラペーニョとハラペーニョとハラペーニョとハラペーニョがトッピングされたピザ



## 3.2.1 旧クラスとPython2 のsuper
- Pyhton2 ではsuperは引数なしでは使え無い
- クラスは明示的にobject や親クラスを引数とした場合に新スタイルクラスとなる

## 3.2.2 Pythonのメソッド解決順序（MRO）
- Python2 は深さ有線探索．もっとも古いクラスのメソッドを継承することに
- Python3 ⇒ C3 線形化アルゴリズム
- それぞれの基底クラスに対して深さ方向への再帰的な探索を行い，クラスが複数のリストに含まれている場合は左方優先のルールにのっとる．


In [38]:
class CommonBase:
    def method(self):
        print("CommonBase")
class Base1(CommonBase):
    pass
class Base2(CommonBase):
    def method(self):
        print("Base2")
class MyClass(Base1,Base2):
    pass

MyClass().method()

Base2


In [40]:
def L(some_class):
    return [k.__name__ for k in some_class.__mro__]
L(MyClass)

['MyClass', 'Base1', 'Base2', 'CommonBase', 'object']

## 3.3.3 superの落とし穴
基底クラスの__init__ が暗黙的に呼び出されない

In [57]:
# 従来の呼び出しとの混在

class A :
    def __init__(self):
        print("A",end=" ")
        super().__init__()
class B:
    def __init__(self):
        print("B",end=" ")
        super().__init__()
        
class C(A,B):
    def __init__(self):
        print("C",end=" ")
        A.__init__(self)
        B.__init__(self)

# 解決方法 全てのクラス階層でsuper を用いる．
# MRO 順に 親から子へSuper が連鎖して一度の初期ですむ C ⇒ A ⇒ B 
# ソースコード調べるのが大事
#         super(C,self).__init__()

print("MRO",[x.__name__ for x in C.__mro__])

MRO ['C', 'A', 'B', 'object']


In [58]:
"""
A.__init__(self)で呼び出されるクラスAの初期化のSuperでMRO一つ右Bのinitが行われている
B.__init__(self)も含め二回初期化が行われている
"""
C()

C A B 

In [68]:
# 親クラスと異なる引数定義の存在
# 初期化での引数渡し
# 親クラスの__init__()のシグネチャと位置しないためTypeError

class CommonBase:
    def __init__(self):
        print("CommonBase")
        super().__init__()

class Base1(CommonBase):
    def __init__(self):
        print("Base1")
        super().__init__()
 
class Base2(CommonBase):
    def __init__(self,arg):
        print("Base2")
        super().__init__()

class MyClass(Base1,Base2):
    def __init__(self,arg):
        print("MyClass")
        super().__init__(arg)

MyClass(10)

MyClass


TypeError: __init__() takes 1 positional argument but 2 were given

In [70]:
# 魔法の *arg,**kwargをすべてのコンストラクタに適用する
# ただしコードが脆弱になる


class CommonBase:
    def __init__(self,*arg,**kwarg):
        print("CommonBase")
        super().__init__()

class Base1(CommonBase):
    def __init__(self,*arg,**kwarg):
        print("Base1")
        super().__init__(*arg,**kwarg)
 
class Base2(CommonBase):
    def __init__(self,*arg,**kwarg):
        print("Base2")
        super().__init__(*arg,**kwarg)

class MyClass(Base1,Base2):
    def __init__(self,arg):
        print("MyClass")
        super().__init__(arg)

MyClass(10)

MyClass
Base1
Base2
CommonBase


# 3.2.4 ベストプラクティス
## 1. 多重継承を避ける
## 2. Super の使用に一貫性を持たせる
## 3. Python2 もターゲットにするときはObjectも明示的に継承する 
## 4. 親クラスを呼ぶときはクラス階層を確認する（MRO）